Kod složenijih primjera `split()` metoda brzo postaje neučinkovita. Dohvatimo linkove pomoću regularnih izraza.

> ZADATAK 3. Dohvaćanje linkova korištenjem biblioteke [re](https://docs.python.org/3/library/re.html); primjer: 

`<a href="/images/nastava/izvedbeni/2023_2024/PDS/1/DINP_FIDIT_2023_2024_OPROG.pdf" target="_blank"><i class="fa fa-file-pdf-o"></i></a></td>`



Metode `re.search()`, `re.match()`, `re.fullmatch()`:
- [re.match()](https://docs.python.org/3/library/re.html#re.match) checks for a match only at the beginning of the string
- [re.search()](https://docs.python.org/3/library/re.html#re.search) checks for a match anywhere in the string (this is what Perl does by default)
- [re.fullmatch()](https://docs.python.org/3/library/re.html#re.fullmatch) checks for entire string to be a match

- [re.compile()](https://docs.python.org/3/library/re.html#re.compile) compiles regular expression

- [re.findall()](https://docs.python.org/3/library/re.html#re.findall) returns all non-overlapping matches of pattern in string, as a list of strings or tuples. 


> ZADATAK 3a Sa Wikipedia članka o metodici [MIRIS](https://hr.wikipedia.org/wiki/MIRIS_(ra%C4%8Dunarstvo)) saznajte:

>> Koliko puta se pojavljuje riječ MIRIS?

>> Koliko puta se spominje prezime koje završava na "ić"? Spremite rezultat u listu.

>> Izlučite sve linkove sa članka.

In [24]:
import requests

url = "https://hr.wikipedia.org/wiki/MIRIS_(ra%C4%8Dunarstvo)"

page = requests.get(url)

print(page.text)

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-disabled skin-theme-clientpref-day vector-toc-available" lang="hr" dir="ltr">
<head>
<meta charset="UTF-8">
<title>MIRIS (računarstvo) – Wikipedija</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 ve

In [7]:
import re

# Pojavljivanje riječi MIRIS

result31 = re.findall(r"MIRIS", page.text)

print(f"Riječ MIRIS pojavljuje se: {len(result31)} puta")

Riječ MIRIS pojavljuje se: 92 puta


In [23]:
# Pojavljivanje ić prezimena

result32 = re.findall(r"\s*([A-Za-zČĆŠĐŽčćšđž]+ić)[ \.\,]", page.text)
print("U tekstu se prezime na \"ić\" pojavljuje: {} puta".format(len(result32)))

U tekstu se prezime na "ić" pojavljuje: 19 puta


In [34]:
# Izlučivanje linkova

regex33 = re.compile(r"(https:\/\/([\w\d-]+\.|[\w\d-]+\/)+)")
result33 = re.findall(regex33, page.text)

print([r[0] for r in result33])

['https://upload.wikimedia.org/wikipedia/commons/5/50/2_Miris-color.', 'https://upload.wikimedia.org/wikipedia/commons/5/50/2_Miris-color.', 'https://hr.wikipedia.org/wiki/', 'https://creativecommons.org/licenses/by-sa/4.0/deed.', 'https://www.wikidata.org/wiki/', 'https://milepavlic.blogspot.com/2017/', 'https://urn.nsk.hr/', 'https://urn.nsk.hr/', 'https://inf.uniri.hr/', 'https://inf.uniri.hr/', 'https://milepavlic.blogspot.com/2017/', 'https://login.wikimedia.org/wiki/', 'https://hr.wikipedia.org/w/index.', 'https://hr.wikipedia.org/w/index.', 'https://creativecommons.org/licenses/by-sa/4.0/deed.', 'https://foundation.wikimedia.org/wiki/', 'https://foundation.wikimedia.org/wiki/', 'https://foundation.wikimedia.org/wiki/', 'https://developer.wikimedia.', 'https://stats.wikimedia.org/', 'https://foundation.wikimedia.org/wiki/', 'https://wikimediafoundation.org/', 'https://www.mediawiki.org/']


Izlučivanje informacija relativno je komplesan zadatak, posebice kada govorimo o ciljevima kao što su URL, adresa e-pošte ili imena osoba.

Preuzmimo izvedbene programe [Fakulteta za Fiziku](https://www.phy.uniri.hr/hr/).

Svi izvedbeni programi nalaze se na stranici: https://www.phy.uniri.hr/hr/nastava/izvedbeni-planovi.html .

In [6]:
import requests

URL_DINP = "https://www.phy.uniri.hr/hr/nastava/izvedbeni-planovi.html"

page_dinp = requests.get(URL_DINP)

print(page_dinp.text)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="hr-hr" lang="hr-hr" >
<head>
  <base href="https://www.phy.uniri.hr/hr/nastava/izvedbeni-planovi.html" />
  <meta http-equiv="content-type" content="text/html; charset=utf-8" />
  <meta name="author" content="Klaudija Lončarić" />
  <title>Fakultet za fiziku - Izvedbeni planovi</title>
  <link href="/templates/uniri/favicon.ico" rel="shortcut icon" type="image/vnd.microsoft.icon" />
  <link rel="stylesheet" href="/media/mod_languages/css/template.css" type="text/css" />
  <script src="/media/system/js/caption.js" type="text/javascript"></script>
  <script type="text/javascript">
window.addEvent('load', function() {
				new JCaption('img.caption');
			});
  </script>

<link rel="shortcut icon" href="/templates/uniri/images/favicon.ico" />
<link rel="stylesheet" href="/templates/system/css/system.css" type="text/css

In [7]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(page_dinp.text, 'html.parser')

print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="hr-hr" xml:lang="hr-hr" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <base href="https://www.phy.uniri.hr/hr/nastava/izvedbeni-planovi.html"/>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="Klaudija Lončarić" name="author"/>
  <title>
   Fakultet za fiziku - Izvedbeni planovi
  </title>
  <link href="/templates/uniri/favicon.ico" rel="shortcut icon" type="image/vnd.microsoft.icon"/>
  <link href="/media/mod_languages/css/template.css" rel="stylesheet" type="text/css"/>
  <script src="/media/system/js/caption.js" type="text/javascript">
  </script>
  <script type="text/javascript">
   window.addEvent('load', function() {
				new JCaption('img.caption');
			});
  </script>
  <link href="/templates/uniri/images/favicon.ico" rel="shortcut icon"/>
  <link href="/templates/system/css/system.css" rel="stylesheet" type

In [8]:
linkovi_dinp = []
retci = soup.find_all('td')
for r in retci:
    try:
        print(r.find('a')["href"])
        linkovi_dinp.append(r.find('a')["href"])
    except:
        pass

/files/nastava/izvedbeni_programi/prijedipl_fizika/Algoritmi i strukture podataka 2024.pdf
/files/nastava/izvedbeni_programi/prijedipl_fizika/Baze podataka 2024.pdf
/files/nastava/izvedbeni_programi/prijedipl_fizika/Diferencijalne jednadzbe izvedbeni 2024.pdf
/files/nastava/izvedbeni_programi/prijedipl_fizika/Elektrodinamika_izvedbeni_2024.pdf
/files/nastava/izvedbeni_programi/prijedipl_fizika/Engleski jezik u struci izvedbeni 2024.pdf
/files/nastava/izvedbeni_programi/prijedipl_fizika/Euklidski prostori 2024.pdf
/files/nastava/izvedbeni_programi/prijedipl_fizika/Fizicki praktikum I izvedbeni 2024.pdf
/files/nastava/izvedbeni_programi/prijedipl_fizika/Fizicki praktikum II izvedbeni 2024.pdf
/files/nastava/izvedbeni_programi/prijedipl_fizika/Fizicki praktikum III izvedbeni 2024.pdf
/files/nastava/izvedbeni_programi/prijedipl_fizika/Fizicki_praktikum4_izvedbeni_2024.pdf
/files/nastava/izvedbeni_programi/prijedipl_fizika/Fizika I izvedbeni 2024.pdf
/files/nastava/izvedbeni_programi/prijed